In [1]:
import numpy as np
import pandas as pd
import sklearn

In [39]:
# load datasets
btc_data = pd.read_csv('BTC-USD.csv')
ndx_data = pd.read_csv('^NDX.csv')
btc_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,7200.174316,18565664997
1,2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,6985.470215,20802083465
2,2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,7344.884277,28111481032
3,2020-01-04,7345.375488,7427.385742,7309.514160,7410.656738,7410.656738,18444271275
4,2020-01-05,7410.451660,7544.497070,7400.535645,7411.317383,7411.317383,19725074095


In [42]:
btc_change = (btc_data['Close'] - btc_data['Open']) / btc_data['Close'] * 100
btc_data['btc_pct_change'] = btc_change

ndx_change = (ndx_data['Close'] - ndx_data['Open']) / ndx_data['Close'] * 100
ndx_data['ndx_pct_change'] = ndx_change

# shift 'Close' up by 1 to construct labels:
btc_data['shifted_close'] = btc_data['Close'].shift(-1)
btc_data['label'] = np.where(btc_data['shifted_close'] - btc_data['Close'] > 0, 1, 0)

In [44]:
btc_data = btc_data.join(ndx_data.set_index('Date'), on='Date', rsuffix='_ndx')
btc_data = btc_data.dropna(axis=0)

In [48]:
pruned = btc_data.loc[:, ['btc_pct_change', 'ndx_pct_change', 'Volume']]
labels = btc_data.loc[:, 'label']

In [52]:
# split into train and validation
X = pruned.to_numpy()
y = labels.to_numpy()

X_train = X[:450]
X_val = X[451:]

y_train = y[:450]
y_val = y[451:]

In [54]:
from sklearn import tree
dec_tree = tree.DecisionTreeClassifier()
dec_tree.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [55]:
from sklearn.metrics import accuracy_score
y_preds = dec_tree.predict(X_val)
score = accuracy_score(y_val, y_preds)
score

0.5404255319148936